In [ ]:
!pip install torch torchvision ultralytics opencv-python
!pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Data preprocessing - vykreslenie ciernych rectangles pre zadefinovanu stanicu na zakrytie statickych objektov po obvode obrazu, ktore si model casto myli

In [ ]:
import os
import cv2
import glob

test_images_path = r"C:\Users\livia\Desktop\DP_rozdelenie_dat\data\test\images"
output_folder = r"C:\Users\livia\Desktop\DP_rozdelenie_dat\data\test\images_preprocessed"

# Vytvorenie priečinka na uloženie upravených obrázkov
os.makedirs(output_folder, exist_ok=True)

# Načítanie zoznamu obrázkov
image_paths = glob.glob(os.path.join(test_images_path, "*.jpg"))

# Spracovanie obrázkov
for img_path in image_paths:
    filename = os.path.basename(img_path)
    img = cv2.imread(img_path)

    if img is None:
        print(f"❌ Chyba pri načítaní: {img_path}")
        continue

    # AMOS stanica "AMOS-CE_P"
    if "AMOS-CE_P" in filename:
        rect_start = (140, 200)
        rect_end = (220, 330)
        cv2.rectangle(img, rect_start, rect_end, (0, 0, 0), thickness=-1)

    # AMOS stanica "FO_P" 
    if "FO_P" in filename:
        rect1_start = (80, 100)
        rect1_end = (160, 190)
        rect2_start = (90, 985)
        rect2_end = (150, 1030)
        cv2.rectangle(img, rect1_start, rect1_end, (0, 0, 0), thickness=-1)
        cv2.rectangle(img, rect2_start, rect2_end, (0, 0, 0), thickness=-1)

    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, img)

print(f"✅ Preprocessing hotový! Upravené obrázky uložené v: {output_folder}")

Vykreslenie bboxov na testovacej mnozine s predikciami + nastavenie thresholdu (31%)

In [ ]:
import torch
import cv2
import os
import glob
import pandas as pd
import shutil
from ultralytics import YOLO
from PIL import Image

# Nastavenie ciest
model_path = "yolov11_weights.pt" 
test_images_path = r"C:\Users\livia\Desktop\DP_rozdelenie_dat\data\test\images_preprocessed"
output_folder = r"C:\Users\livia\Desktop\DP_rozdelenie_dat\data\test\results\yolov11_preprocessed_thresh_above31"
excel_path = os.path.join(output_folder, "meteor_detections.xlsx")


os.makedirs(output_folder, exist_ok=True)
model = YOLO(model_path)

image_paths = glob.glob(os.path.join(test_images_path, "*.jpg"))

# pre Excel
detections = []

# Spracovanie obrázkov
for img_path in image_paths:
    img = cv2.imread(img_path)  # Načítanie obrázka cez OpenCV
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # YOLO očakáva RGB
    filename = os.path.basename(img_path)  # Extrahovanie názvu súboru

    # Spustenie modelu na obrázku
    results = model(img_rgb)

    # Flag na uloženie obrázka, ak je aspoň jedna detekcia nad prahom
    save_image = False

    # Spracovanie výstupu
    for result in results:
        if result.boxes is not None:  # Overenie, že sú detekované objekty
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box súradnice
                conf = box.conf[0].item()  # Pravdepodobnosť predikcie
                cls_id = int(box.cls[0].item())  # Trieda objektu

                # Overenie, či model detegoval "Meteor" a istota je nad 30 %
                if cls_id == 0 and conf > 0.31:  # Predpokladám, že "Meteor" je class_id = 0
                    # Vykreslenie bounding boxu
                    color = (0, 255, 0)
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                    # Pridanie textu s predikciou
                    text = f"Meteor: {conf:.2f}"
                    cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Uloženie informácií do zoznamu pre Excel
                    detections.append({
                        "Image": filename,
                        "Confidence": conf,
                        "BoundingBox": f"({x1}, {y1}) - ({x2}, {y2})"
                    })

                    save_image = True

    # Uloženie obrázka s detekciami alebo jeho prekopírovanie, ak neobsahuje meteory
    output_path = os.path.join(output_folder, filename)
    
    if save_image:
        cv2.imwrite(output_path, img)
    else:
        shutil.copy(img_path, output_path)

# Uloženie výsledkov do Excelu
if detections:
    df = pd.DataFrame(detections)
    df.to_excel(excel_path, index=False)
    print(f"✅ Výsledky uložené v: {output_folder}")
    print(f"📄 Excel súbor s detekciami uložený: {excel_path}")
else:
    print("❌ Neboli detegované žiadne meteory s pravdepodobnosťou nad 31 %.")



0: 864x1120 1 Meteor, 40.7ms
Speed: 11.4ms preprocess, 40.7ms inference, 2.5ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 1 Meteor, 37.1ms
Speed: 12.6ms preprocess, 37.1ms inference, 1.9ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 2 Meteors, 37.0ms
Speed: 13.3ms preprocess, 37.0ms inference, 3.2ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 1 Meteor, 62.4ms
Speed: 19.5ms preprocess, 62.4ms inference, 5.3ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 3 Meteors, 36.9ms
Speed: 19.7ms preprocess, 36.9ms inference, 2.2ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 1 Meteor, 37.1ms
Speed: 14.4ms preprocess, 37.1ms inference, 3.9ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 1 Meteor, 37.2ms
Speed: 13.5ms preprocess, 37.2ms inference, 3.2ms postprocess per image at shape (1, 3, 864, 1120)

0: 864x1120 1 Meteor, 36.7ms
Speed: 19.0ms preprocess, 36.7ms inference, 3.2ms postprocess per